## 1. The oldest business in the world
<p><img src="https://assets.datacamp.com/production/repositories/5851/datasets/7dded924c6dc418d4a828f2f4daba99953c27a5a/400px-Eingang_zum_St._Peter_Stiftskeller.jpg" alt="The entrance to St. Peter Stiftskeller, a restaurant in Saltzburg, Austria. The sign over the entrance shows &quot;803&quot; - the year the business opened."></p>
<p><em>Image: St. Peter Stiftskeller, founded 803. Credit: <a href="https://commons.wikimedia.org/wiki/File:Eingang_zum_St._Peter_Stiftskeller.jpg">Pakeha</a>.</em></p>
<p>An important part of business is planning for the future and ensuring that the company survives changing market conditions. Some businesses do this really well and last for hundreds of years.</p>
<p>BusinessFinancing.co.uk <a href="https://businessfinancing.co.uk/the-oldest-company-in-almost-every-country">researched</a> the oldest company that is still in business in (almost) every country and compiled the results into a dataset. In this project, you'll explore that dataset to see what they found.</p>
<p>The database contains three tables.</p>
<h3 id="categories"><code>categories</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>category_code</code></td>
<td>varchar</td>
<td>Code for the category of the business.</td>
</tr>
<tr>
<td style="text-align:left;"><code>category</code></td>
<td>varchar</td>
<td>Description of the business category.</td>
</tr>
</tbody>
</table>
<h3 id="countries"><code>countries</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>country_code</code></td>
<td>varchar</td>
<td>ISO 3166-1 3-letter country code.</td>
</tr>
<tr>
<td style="text-align:left;"><code>country</code></td>
<td>varchar</td>
<td>Name of the country.</td>
</tr>
<tr>
<td style="text-align:left;"><code>continent</code></td>
<td>varchar</td>
<td>Name of the continent that the country exists in.</td>
</tr>
</tbody>
</table>
<h3 id="businesses"><code>businesses</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>business</code></td>
<td>varchar</td>
<td>Name of the business.</td>
</tr>
<tr>
<td style="text-align:left;"><code>year_founded</code></td>
<td>int</td>
<td>Year the business was founded.</td>
</tr>
<tr>
<td style="text-align:left;"><code>category_code</code></td>
<td>varchar</td>
<td>Code for the category of the business.</td>
</tr>
<tr>
<td style="text-align:left;"><code>country_code</code></td>
<td>char</td>
<td>ISO 3166-1 3-letter country code.</td>
</tr>
</tbody>
</table>
<p>Let's begin by looking at the range of the founding years throughout the world.</p>

In [65]:
%%sql 
postgresql:///oldestbusinesses
select min(year_founded), max(year_founded)
from businesses

1 rows affected.


[(578, 1999)]

## 2. How many businesses were founded before 1000?
<p>Wow! That's a lot of variation between countries. In one country, the oldest business was only founded in 1999. By contrast, the oldest business in the world was founded back in 578. That's pretty incredible that a business has survived for more than a millennium.</p>
<p>I wonder how many other businesses there are like that.</p>

In [67]:
%%sql
select count(*)
from businesses
where year_founded < 1000

 * postgresql:///oldestbusinesses
1 rows affected.


[(6,)]

## 3. Which businesses were founded before 1000?
<p>Having a count is all very well, but I'd like more detail. Which businesses have been around for more than a millennium?</p>

In [69]:
%%sql

select *
from businesses
where year_founded < 1000
order by year_founded;

 * postgresql:///oldestbusinesses
6 rows affected.


[('Kongō Gumi', 578, 'CAT6', 'JPN'),
 ('St. Peter Stifts Kulinarium', 803, 'CAT4', 'AUT'),
 ('Staffelter\xa0Hof Winery', 862, 'CAT9', 'DEU'),
 ('Monnaie de Paris\xa0', 864, 'CAT12', 'FRA'),
 ('The Royal Mint', 886, 'CAT12', 'GBR'),
 ("Sean's Bar", 900, 'CAT4', 'IRL')]

## 4. Exploring the categories
<p>Now we know that the oldest, continuously operating company in the world is called Kongō Gumi. But was does that company do? The category codes in the <code>businesses</code> table aren't very helpful: the descriptions of the categories are stored in the <code>categories</code> table.</p>
<p>This is a common problem: for data storage, it's better to keep different types of data in different tables, but for analysis, you want all the data in one place. To solve this, you'll have to join the two tables together.</p>

In [71]:
%%sql


select b.business , b.year_founded , b.country_code ,c.category 
from businesses b
join categories c
ON b.category_code = c.category_code
where year_founded < 1000
order by year_founded

 * postgresql:///oldestbusinesses
6 rows affected.


[('Kongō Gumi', 578, 'JPN', 'Construction'),
 ('St. Peter Stifts Kulinarium', 803, 'AUT', 'Cafés, Restaurants & Bars'),
 ('Staffelter\xa0Hof Winery', 862, 'DEU', 'Distillers, Vintners, & Breweries'),
 ('Monnaie de Paris\xa0', 864, 'FRA', 'Manufacturing & Production'),
 ('The Royal Mint', 886, 'GBR', 'Manufacturing & Production'),
 ("Sean's Bar", 900, 'IRL', 'Cafés, Restaurants & Bars')]

## 5. Counting the categories
<p>With that extra detail about the oldest businesses, we can see that Kongō Gumi is a construction company. In that list of six businesses, we also see a café, a winery, and a bar. The two companies recorded as "Manufacturing and Production" are both mints. That is, they produce currency.</p>
<p>I'm curious as to what other industries constitute the oldest companies around the world, and which industries are most common.</p>

In [73]:
%%sql

SELECT c.category, count(c.category) n
    FROM categories c
    JOIN businesses b
        ON b.category_code = c.category_code
    GROUP BY category
    ORDER BY count(c.category) desc
    LIMIT 10;

 * postgresql:///oldestbusinesses
10 rows affected.


[('Banking & Finance', 37),
 ('Distillers, Vintners, & Breweries', 22),
 ('Aviation & Transport', 19),
 ('Postal Service', 16),
 ('Manufacturing & Production', 15),
 ('Media', 7),
 ('Agriculture', 6),
 ('Cafés, Restaurants & Bars', 6),
 ('Food & Beverages', 6),
 ('Tourism & Hotels', 4)]

## 6. Oldest business by continent
<p>It looks like "Banking &amp; Finance" is the most popular category. Maybe that's where you should aim if you want to start a thousand-year business.</p>
<p>One thing we haven't looked at yet is where in the world these really old businesses are. To answer these questions, we'll need to join the <code>businesses</code> table to the <code>countries</code> table. Let's start by asking how old the oldest business is on each continent.</p>

In [75]:
%%sql


select min(b.year_founded) oldest, c.continent
from businesses b
join countries c 
on b.country_code = c.country_code
group by c.continent
order by oldest

 * postgresql:///oldestbusinesses
6 rows affected.


[(578, 'Asia'),
 (803, 'Europe'),
 (1534, 'North America'),
 (1565, 'South America'),
 (1772, 'Africa'),
 (1809, 'Oceania')]

## 7. Joining everything for further analysis
<p>Interesting. There's a jump in time from the older businesses in Asia and Europe to the 16th Century oldest businesses in North and South America, then to the 18th and 19th Century oldest businesses in Africa and Oceania. </p>
<p>As mentioned earlier, when analyzing data it's often really helpful to have all the tables you want access to joined together into a single set of results that can be analyzed further. Here, that means we need to join all three tables.</p>

In [77]:
%%sql


select b.business , b.year_founded ,c.category , cn.country , cn.continent
from businesses b 
join categories c 
on b.category_code = c.category_code
join countries cn
on b.country_code = cn.country_code

 * postgresql:///oldestbusinesses
163 rows affected.


[('Spinzar Cotton Company', 1930, 'Agriculture', 'Afghanistan', 'Asia'),
 ('ALBtelecom', 1912, 'Telecommunications', 'Albania', 'Europe'),
 ('Andbank', 1930, 'Banking & Finance', 'Andorra', 'Europe'),
 ('Liwa Chemicals', 1939, 'Manufacturing & Production', 'United Arab Emirates', 'Asia'),
 ('Bank of the Province of Buenos Aires', 1822, 'Banking & Finance', 'Argentina', 'South America'),
 ('Yerevan Ararat Brandy-Wine-Vodka Factory', 1877, 'Distillers, Vintners, & Breweries', 'Armenia', 'Asia'),
 ('Australia Post', 1809, 'Postal Service', 'Australia', 'Oceania'),
 ('St. Peter Stifts Kulinarium', 803, 'Cafés, Restaurants & Bars', 'Austria', 'Europe'),
 ('Azerbaijan Caspian Shipping Company', 1858, 'Aviation & Transport', 'Azerbaijan', 'Asia'),
 ('Brarudi', 1955, 'Distillers, Vintners, & Breweries', 'Burundi', 'Africa'),
 ('Affligem Brewery', 1074, 'Distillers, Vintners, & Breweries', 'Belgium', 'Europe'),
 ('Communauté Électrique du Bénin', 1968, 'Energy', 'Benin', 'Africa'),
 ('Air Burkina', 1967, 'Aviation & Transport', 'Burkina Faso', 'Africa'),
 ('M. M. Ispahani Limited', 1820, 'Food & Beverages', 'Bangladesh', 'Asia'),
 ('Arsenal AD', 1878, 'Defense', 'Bulgaria', 'Europe'),
 ('BMMI', 1883, 'Retail', 'Bahrain', 'Asia'),
 ('Sarajevska Pivara', 1864, 'Distillers, Vintners, & Breweries', 'Bosnia and Herzegovina', 'Europe'),
 ('Olivaria Brewery', 1864, 'Distillers, Vintners, & Breweries', 'Belarus', 'Europe'),
 ('Belize Bank', 1902, 'Banking & Finance', 'Belize', 'North America'),
 ('Banco Nacional de Bolivia', 1871, 'Banking & Finance', 'Bolivia, Plurinational State of', 'South America'),
 ('Casa da Moeda do Brasil', 1694, 'Manufacturing & Production', 'Brazil', 'South America'),
 ('Mount Gay Rum', 1703, 'Distillers, Vintners, & Breweries', 'Barbados', 'North America'),
 ('Hua Ho Department Store', 1947, 'Retail', 'Brunei Darussalam', 'Asia'),
 ('Tashi Group', 1959, 'Aviation & Transport', 'Bhutan', 'Asia'),
 ('Botswana Meat Commission', 1965, 'Agriculture', 'Botswana', 'Africa'),
 ('Banque Internationale pour la Centrafrique', 1946, 'Banking & Finance', 'Central African Republic', 'Africa'),
 ("Hudson's Bay Company", 1670, 'Retail', 'Canada', 'North America'),
 ('Gasthof Sternen', 1230, 'Tourism & Hotels', 'Switzerland', 'Europe'),
 ('Famae', 1811, 'Defense', 'Chile', 'South America'),
 ("Ma Yu Ching's Bucket Chicken House", 1153, 'Cafés, Restaurants & Bars', 'China', 'Asia'),
 ('La Poste', 1945, 'Postal Service', "Côte d'Ivoire", 'Africa'),
 ('Cameroon Development Corporation', 1947, 'Agriculture', 'Cameroon', 'Africa'),
 ('Société nationale des Chemins de fer du Congo', 1889, 'Aviation & Transport', 'Congo, The Democratic Republic of the', 'Africa'),
 ('Development Bank of the Central African States', 1975, 'Banking & Finance', 'Congo', 'Africa'),
 ('Casa de Moneda de Colombia', 1621, 'Manufacturing & Production', 'Colombia', 'South America'),
 ('Central Bank of the Comoros', 1981, 'Banking & Finance', 'Comoros', 'Africa'),
 ('Correios de Cabo Verde', 1849, 'Postal Service', 'Cabo Verde', 'Africa'),
 ('Florida Ice and Farm Company', 1908, 'Distillers, Vintners, & Breweries', 'Costa Rica', 'North America'),
 ('Cubana de Aviación', 1929, 'Aviation & Transport', 'Cuba', 'North America'),
 ('Bank of Cyprus', 1899, 'Banking & Finance', 'Cyprus', 'Europe'),
 ('Pivovar Broumov', 1348, 'Distillers, Vintners, & Breweries', 'Czechia', 'Europe'),
 ('Staffelter\xa0Hof Winery', 862, 'Distillers, Vintners, & Breweries', 'Germany', 'Europe'),
 ('Ethio-Djibouti Railways', 1901, 'Aviation & Transport', 'Djibouti', 'Africa'),
 ('The Chronicle (Dominica)', 1909, 'Media', 'Dominica', 'North America'),
 ('Munke Mølle', 1135, 'Manufacturing & Production', 'Denmark', 'Europe'),
 ('Hamoud Boualem', 1878, 'Food & Beverages', 'Algeria', 'Africa'),
 ('Egyptian National Railways', 1854, 'Aviation & Transport', 'Egypt', 'Africa'),
 ('Asmara Brewery', 1939, 'Distillers, Vintners, & Breweries', 'Eritrea', 'Africa'),
 ('Casa de Ganaderos', 1218, 'Agricultur

## 8. Counting categories by continent
<p>Having <code>businesses</code> joined to <code>categories</code> and <code>countries</code> together means we can ask questions about both these things together. For example, which are the most common categories for the oldest businesses on each continent?</p>

In [79]:
%%sql


SELECT cnt.continent, cat.category, COUNT(*) AS n
    FROM businesses AS bus
    INNER JOIN categories as cat
        ON bus.category_code = cat.category_code
    INNER JOIN countries as cnt
        ON bus.country_code = cnt.country_code
    GROUP BY cnt.continent, cat.category;

 * postgresql:///oldestbusinesses
56 rows affected.


[('North America', 'Banking & Finance', 4),
 ('Oceania', 'Postal Service', 1),
 ('South America', 'Food & Beverages', 2),
 ('Europe', 'Tourism & Hotels', 2),
 ('Asia', 'Media', 1),
 ('Europe', 'Medical', 1),
 ('Asia', 'Defense', 1),
 ('Africa', 'Manufacturing & Production', 1),
 ('Europe', 'Postal Service', 4),
 ('North America', 'Aviation & Transport', 2),
 ('Asia', 'Distillers, Vintners, & Breweries', 2),
 ('South America', 'Banking & Finance', 3),
 ('North America', 'Food & Beverages', 1),
 ('Europe', 'Manufacturing & Production', 8),
 ('Africa', 'Postal Service', 9),
 ('Asia', 'Telecommunications', 1),
 ('Africa', 'Food & Beverages', 1),
 ('Europe', 'Consumer Goods', 3),
 ('Europe', 'Mining', 1),
 ('Oceania', 'Banking & Finance', 2),
 ('Asia', 'Agriculture', 1),
 ('North America', 'Manufacturing & Production', 1),
 ('Africa', 'Mining', 1),
 ('Africa', 'Aviation & Transport', 10),
 ('Asia', 'Construction', 2),
 ('Asia', 'Energy', 3),
 ('Asia', 'Retail', 3),
 ('South America', 'Manufacturing & Production', 2),
 ('Asia', 'Cafés, Restaurants & Bars', 3),
 ('Europe', 'Banking & Finance', 5),
 ('Africa', 'Banking & Finance', 17),
 ('North America', 'Tourism & Hotels', 1),
 ('Europe', 'Cafés, Restaurants & Bars', 2),
 ('Asia', 'Banking & Finance', 6),
 ('South America', 'Defense', 1),
 ('Africa', 'Energy', 1),
 ('Asia', 'Conglomerate', 3),
 ('North America', 'Media', 1),
 ('Europe', 'Agriculture', 1),
 ('Asia', 'Mining', 1),
 ('Asia', 'Aviation & Transport', 7),
 ('Asia', 'Food & Beverages', 2),
 ('North America', 'Distillers, Vintners, & Breweries', 5),
 ('Africa', 'Agriculture', 3),
 ('Africa', 'Distillers, Vintners, & Breweries', 3),
 ('North America', 'Agriculture', 1),
 ('Asia', 'Postal Service', 2),
 ('Europe', 'Defense', 1),
 ('Asia', 'Manufacturing & Production', 3),
 ('Europe', 'Telecommunications', 1),
 ('Europe', 'Distillers, Vintners, & Breweries', 12),
 ('South America', 'Cafés, Restaurants & Bars', 1),
 ('North America', 'Retail', 1),
 ('Africa', 'Media', 4),
 ('Europe', 'Media', 1),
 ('Asia', 'Tourism & Hotels', 1)]

## 9. Filtering counts by continent and category
<p>Combining continent and business category led to a lot of results. It's difficult to see what is important. To trim this down to a manageable size, let's restrict the results to only continent/category pairs with a high count.</p>

In [81]:
%%sql

SELECT cnt.continent, cat.category, COUNT(*) AS n
    FROM businesses AS bus
    INNER JOIN categories as cat
        ON bus.category_code = cat.category_code
    INNER JOIN countries as cnt
        ON bus.country_code = cnt.country_code
    GROUP BY cnt.continent, cat.category
    having count(*) > 5
    order by n desc

 * postgresql:///oldestbusinesses
7 rows affected.


[('Africa', 'Banking & Finance', 17),
 ('Europe', 'Distillers, Vintners, & Breweries', 12),
 ('Africa', 'Aviation & Transport', 10),
 ('Africa', 'Postal Service', 9),
 ('Europe', 'Manufacturing & Production', 8),
 ('Asia', 'Aviation & Transport', 7),
 ('Asia', 'Banking & Finance', 6)]